In [75]:
import os

In [76]:
%pwd

'C:\\Users\\PC\\Desktop\\work\\projectML\\Chicken-Disease-Classification'

In [77]:
os.chdir(r"C:\Users\PC\Desktop\work\projectML\Chicken-Disease-Classification")

In [78]:
%pwd

'C:\\Users\\PC\\Desktop\\work\\projectML\\Chicken-Disease-Classification'

In [79]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file: Path
    unzip_dir: Path

In [80]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [81]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,     
            params_filepath = PARAMS_FILE_PATH     
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return  data_ingestion_config

In [82]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

import socket


In [83]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        
        if not os.path.exists(self.config.local_data_file):
                
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename= self.config.local_data_file
             )
                logger.info(f"{filename}  download! with following info : \n{headers}")
        else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Fonction returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        
        

In [85]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-21 20:23:47,181: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-21 20:23:47,182: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-21 20:23:47,182: INFO: common: created directory at: artifacts]
[2025-03-21 20:23:47,183: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-21 20:23:58,270: INFO: 2907382464: artifacts/data_ingestion/data.zip  download! with following info : 
Connection: close
Content-Length: 11615540
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d6005e5df3d9411bed8e26939d250636818014d9f4bd158cc92ccd5d1a7ef9ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7848:3DBAC8:1FB3C8:234049:67DDBCC1
Accept-Ranges: bytes
Date: Fri, 21 Mar 2025 19:23:48 GMT
Via: 1.1 varnish
X-Served-By: cache-mxp6944-MXP
X-Cache: